In [ ]:
import logging
import time
import ray
import random
from random import randint
import numpy as np

In [ ]:
from collections import defaultdict
@ray.remote
class LoggingActor(object):
    def __init__(self):
        # create a container of dictionaries.
        self.logs = defaultdict(list)
    
    # log the message for a particular experiment in its 
    # respective dictionary
    def log(self, index, message):
        self.logs[index].append(message)
    
    # fetch all logs as collection
    def get_logs(self):
        return dict(self.logs)
    
@ray.remote
def run_experiment(experiment_index, logging_actor):
    for i in range(9):
        # pretend this is an experiment that produces a nine results for 
        # experiment result; in our case it's just a simple message 
        # Push a logging message to the actor.
        exp_key = f"experiment-{experiment_index}"
        logging_actor.log.remote(exp_key, 'On iteration {}'.format(i))  

In [ ]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

Run three experiments.

In [ ]:
logging_actor = LoggingActor.remote()
experiment_ids = []
# Run three different experiments
for i in range(3):
    experiment_ids.append(run_experiment.remote(i, logging_actor))

### Fetch the results 

For each experement, we will have 9 iteration results

In [ ]:
for i in range(3):
    time.sleep(1)
    logs = logging_actor.get_logs.remote()
    print(ray.get(logs))

In [ ]:
ray.shutdown()